In [1]:
!pip install beautifulsoup4

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

KOBIS_API_KEY = os.getenv("KOBIS_API_KEY")

API_MOVIE_LIST = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
API_MOVIE_INFO = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
API_PEOPLE_LIST = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"

PEOPLE_PAGE = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; KOBIS-crawler/1.0)"
})

In [3]:
def get_movie_cd(movie_name: str) -> str | None:
    """영화명으로 movieCd(영화코드) 가져오기"""
    params = {
        "key": KOBIS_API_KEY,
        "movieNm": movie_name,
        "itemPerPage": 10,
    }
    r = session.get(API_MOVIE_LIST, params=params)
    r.raise_for_status()
    data = r.json()
    movies = data.get("movieListResult", {}).get("movieList", [])
    if not movies:
        return None

    # 가장 첫 번째 결과 사용 (필요하면 prdtYear, 국가 등으로 추가 필터링)
    return movies[0]["movieCd"]


def get_main_actors(movie_cd: str, max_actors: int = 10) -> list[dict]:
    """movieCd로 영화 상세에서 배우 목록 중 상위 max_actors명을 '주연'처럼 사용"""
    params = {"key": KOBIS_API_KEY, "movieCd": movie_cd}
    r = session.get(API_MOVIE_INFO, params=params)
    r.raise_for_status()
    data = r.json()

    movie_info = data["movieInfoResult"]["movieInfo"]
    actors = movie_info.get("actors", [])

    # 디버깅용: 어떤 정보가 오는지 한 번 찍어보고 싶으면 주석 해제
    # for a in actors:
    #    print(a)

    # KOBIS API는 '주연/조연' 구분이 없으므로, 상위 N명을 대표 배우로 사용
    return actors[:max_actors]


def get_people_cd(actor_name: str, movie_name: str) -> str | None:
    """
    배우 이름 + 해당 영화 제목으로 영화인 코드(peopleCd) 찾기
    동일 이름 여러 명일 수 있어서 filmoNames 조건도 같이 줌
    """
    params = {
        "key": KOBIS_API_KEY,
        "peopleNm": actor_name,
        "filmoNames": movie_name,
        "itemPerPage": 10,
    }
    r = session.get(API_PEOPLE_LIST, params=params)
    r.raise_for_status()
    data = r.json()

    people_list = data.get("peopleListResult", {}).get("peopleList", [])
    if not people_list:
        return None

    # 가장 잘 맞는 1개만 사용 (필요하면 더 정교한 매칭 로직 작성)
    return people_list[0]["peopleCd"]

def get_actor_images_from_people_page(people_cd: str) -> list[str]:
    """
    영화인 코드(peopleCd)로 KOBIS 모바일 영화인 상세 페이지를 가져와서
    사람 사진(/common/mast/people/) URL들을 추출.
    """
    DETAIL_URL = "https://www.kobis.or.kr/kobis/mobile/mast/peop/searchPeopleDtl.do"

    params = {"peopleCd": people_cd}
    r = session.get(DETAIL_URL, params=params)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    image_urls: list[str] = []

    # 페이지 안의 모든 img 중 사람 사진 경로만 수집
    for img in soup.find_all("img"):
        src = img.get("src") or ""
        if "/common/mast/people/" not in src:
            continue
        full = urljoin(r.url, src)
        if full not in image_urls:
            image_urls.append(full)

    return image_urls

def get_main_actor_images(movie_name: str) -> dict[str, str]:
    """
    영화명 → 배우 → 대표 이미지 1장 URL
    리턴: {배우이름: 대표이미지URL}
    """
    movie_cd = get_movie_cd(movie_name)
    if not movie_cd:
        raise ValueError(f"영화명을 찾을 수 없음: {movie_name}")

    main_actors = get_main_actors(movie_cd)
    result: dict[str, str] = {}

    for actor in main_actors:
        name = actor["peopleNm"]
        print(f"[+] 주연 배우 처리 중: {name}")
        people_cd = get_people_cd(name, movie_name)
        if not people_cd:
            print(f"    -> peopleCd를 찾지 못했습니다.")
            continue

        img_urls = get_actor_images_from_people_page(people_cd)
        if not img_urls:
            continue

        result[name] = img_urls[0]  # 첫 번째 이미지를 대표로 사용

    return result

In [4]:
movie_title = "기생충"
images_by_actor = get_main_actor_images(movie_title)
print(images_by_actor)

[+] 주연 배우 처리 중: 송강호
[+] 주연 배우 처리 중: 이선균
[+] 주연 배우 처리 중: 조여정
[+] 주연 배우 처리 중: 최우식
[+] 주연 배우 처리 중: 박소담
[+] 주연 배우 처리 중: 이정은
[+] 주연 배우 처리 중: 장혜진
[+] 주연 배우 처리 중: 박명훈
[+] 주연 배우 처리 중: 정지소
[+] 주연 배우 처리 중: 정현준
{'송강호': 'https://www.kobis.or.kr/common/mast/people/2017/07/thumb_x110/thn_212ae19d375049fa86ffa72bed520a28.jpg', '이선균': 'https://www.kobis.or.kr/common/mast/people/2019/09/thumb_x110/thn_0891637b67014d9a9d6b47556c2ad355.jpg', '조여정': 'https://www.kobis.or.kr/common/mast/people/2020/02/thumb_x110/thn_352368f0358b4932974e3578330679a6.jpg', '최우식': 'https://www.kobis.or.kr/common/mast/people/2020/01/thumb_x110/thn_1f768834f6834c75b9249328e69a18a9.jpg', '박소담': 'https://www.kobis.or.kr/common/mast/people/2019/10/thumb_x110/thn_489ac59594a84cbf8f6d76238f97fbf2.jpg', '장혜진': 'https://www.kobis.or.kr/common/mast/people/2019/04/thumb_x110/thn_2497de762f914cd7a61204bb4ec7df62.jpg', '박명훈': 'https://www.kobis.or.kr/common/mast/people/2019/10/thumb_x110/thn_c7f78649f03749fba8fe43e227a07e71.jpg', '정지소': '

In [5]:
import os

os.makedirs("actor_images", exist_ok=True)

movie_title = "기생충"
images_by_actor = get_main_actor_images(movie_title)

for actor, url in images_by_actor.items():
    resp = session.get(url)
    resp.raise_for_status()

    ext = os.path.splitext(url)[1] or ".jpg"

    # 파일 이름에서 공백/슬래시 등 위험한 문자 치환해 주는 게 안전함
    safe_actor = actor.replace(" ", "_")
    filename = f"actor_images/{safe_actor}{ext}"

    with open(filename, "wb") as f:
        f.write(resp.content)

    print("saved:", filename)


[+] 주연 배우 처리 중: 송강호
[+] 주연 배우 처리 중: 이선균
[+] 주연 배우 처리 중: 조여정
[+] 주연 배우 처리 중: 최우식
[+] 주연 배우 처리 중: 박소담
[+] 주연 배우 처리 중: 이정은
[+] 주연 배우 처리 중: 장혜진
[+] 주연 배우 처리 중: 박명훈
[+] 주연 배우 처리 중: 정지소
[+] 주연 배우 처리 중: 정현준
saved: actor_images/송강호.jpg
saved: actor_images/이선균.jpg
saved: actor_images/조여정.jpg
saved: actor_images/최우식.jpg
saved: actor_images/박소담.jpg
saved: actor_images/장혜진.jpg
saved: actor_images/박명훈.jpg
saved: actor_images/정지소.jpg
saved: actor_images/정현준.jpg
